In [1]:
import numpy as np
from scipy.linalg import norm 

H_matrix=1/np.sqrt(2)*np.array([[1, 1],
                                [1,-1]])

CNOT_matrix=np.array([[1,0,0,0],
                      [0,1,0,0],
                      [0,0,0,1],
                      [0,0,1,0]])

PAULIX_matrix = np.array([[0,1],
                         [1,0]])

CNOT_tensor = np.reshape(CNOT_matrix, (2,2,2,2))



class State: 
    def __init__(self,n):
        self.n=n
        self.psi=np.zeros((2,)*n) 
        self.psi[(0,)*n]=1

def T(n,a,b,x) :
    m = 2**(n-a)+2**(n-b)
    d = lambda i : 2**(n-x) if (2**(n-x)) & i == 0 else -(2**(n-x))
    T = np.array([([0] * 2**n)] * 2**n)
    for i in range(2**n) :
        for j in range(2**n) :
            T[i][j] = 1 if (i & m == m and j == d(i) + i) or (i & m != m and j == i) else 0
    return T

def X_GATE(i,reg):
    reg.psi = np.tensordot(reg.psi,PAULIX_matrix,(1,i))
    reg.psi=np.moveaxis(reg.psi,0,i)

def HADAMARD(i,reg): 
    reg.psi=np.tensordot(H_matrix,reg.psi,(1,i)) 
    reg.psi=np.moveaxis(reg.psi,0,i)

# def CNOT(target, control, reg):
    
#     reg.psi=np.tensordot(CNOT_tensor, reg.psi, ((2,3),(control,target))) 
#     reg.psi=np.moveaxis(reg.psi,(0,1),(control,target))   

def CNOT(control,target,reg):
     
        control = (control * reg.n) - 1
        target = (target * reg.n) - 1
        prev = reg.psi.shape
        reg.psi = reg.psi.reshape(VAL(reg))
        if reg.psi[control] == 1 and int(reg.psi[target]) == 1:
            reg.psi[target] = 0
        elif reg.psi[control] == 1 and int(reg.psi[target]) == 0:
            reg.psi[target] = 1
        reg.psi = reg.psi.reshape(prev)


def dib(x):
    return int(bin(x)[2:])

def TOFFOLI(reg,inputs):
    if reg.n >= 3:
        try:
            a,b,x = inputs[0],inputs[1],inputs[2]
            toffoli_matrix = T(reg.n,a,b, x)
            inputvalues = str(measure(a,reg)) + str(measure(b,reg)) + str(measure(x,reg))
            
            column_index = int(inputvalues,2)
            row_index = 0
            for row in range(len(toffoli_matrix)):
                if toffoli_matrix[row][column_index] == 1:
                    row_index = row
                    break
                    
            row_index = str(dib(row_index))
            column_index = str(column_index)
            
            while len(row_index) != len(inputvalues):
                row_index  = "0" + row_index
 
            row_index = [int(i) for i in list(row_index)]
            prev_shape = reg.psi.shape

            if reg.n  == 3:
                RESHAPE_TUPLE = (4,2)
            elif reg.n == 4:
                RESHAPE_TUPLE = (4,4)
            elif reg.n == 5:
                RESHAPE_TUPLE = (4,8)
            elif reg.n == 6:
                RESHAPE_TUPLE = (8,8)
        
            reg.psi = reg.psi.flatten().reshape(RESHAPE_TUPLE)
            
            while len(inputs) !=0 :
                p = inputs.pop()
                q = row_index.pop()    
                reg.psi[p][0] = int(q)
            reg.psi = reg.psi.reshape(prev_shape)
        except Exception:
            print("TOFFOLI EXCEPTION")
            pass
    else:
        return - 1

def measure(i,reg): 
    projectors=[ np.array([[1,0],[0,0]]), np.array([[0,0],[0,1]]) ] 
    def project(i,j,reg): 
        projected=np.tensordot(projectors[j],reg.psi,(1,i))
        o = np.moveaxis(projected,0,i)
        return o
    
    projected=project(i,0,reg) 
    if not np.isfinite(projected).all():
        projected = np.array([1/(2 ** (1/2)) for i in projected])
    norm_projected=norm(projected.flatten()) 
  
    if np.random.random()<norm_projected**2: 
        reg.psi=projected/norm_projected
        return 0
    else:
        projected=project(i,1,reg)
        temp = reg.psi
        reg.psi=projected/norm(projected)
        return 1
        
    

def SETVALUE(reg,value,start):
    shape = reg.psi.shape
    reg.psi = reg.psi.reshape(VAL(reg))
    reg.psi[start] = value
    reg.psi = reg.psi.reshape(shape)

def GETVALUE(reg):
    shape = reg.psi.shape
    reg.psi = reg.psi.reshape(VAL(reg))
    answer = reg.psi[0]
    reg.psi = reg.psi.reshape(shape)
    return int(answer)


def VAL(reg):
    res = 1
    for i in reg.psi.shape:
        res *= i
    return res

def measure_all(reg):
    res = ""
    for i in range(reg.n):
        res += str(measure(i,reg))
    return res[::-1]


In [3]:
t1 = State(4)

X_GATE(0,t1)
X_GATE(1,t1)
CNOT(0,2,t1)
CNOT(1,2,t1)
TOFFOLI(t1,[0,1,3])
s = ""
s += str(measure(3,t1))
s += str(measure(1,t1))
s[::-1]

#https://qiskit.org/textbook/ch-states/atoms-computation.html

'10'

In [4]:
def random_number_generation_emulator(emulation_count,register):
    count = 0
    result = []
    while(count <= emulation_count):
        s = ''
        for i in range(register.n):
            HADAMARD(i,register)
        for i in range(register.n):
            s += str(measure(i,register))
        count += 1
        result.append(int(s, 2))
    return result

In [5]:
state = State(16)
random_number_generation_emulator(256,register=state)

[53502,
 6327,
 55510,
 30880,
 16157,
 16725,
 5571,
 19020,
 53632,
 17366,
 38966,
 6211,
 4006,
 2122,
 27246,
 29584,
 10809,
 32018,
 1391,
 59115,
 56584,
 1586,
 31295,
 14801,
 9607,
 8704,
 9115,
 24183,
 25890,
 58880,
 16324,
 23953,
 40560,
 42551,
 15914,
 56403,
 57535,
 38904,
 59842,
 19841,
 38991,
 11979,
 56521,
 46839,
 51896,
 53417,
 22971,
 2247,
 15737,
 56276,
 18271,
 29273,
 10589,
 297,
 53525,
 44628,
 13380,
 56002,
 16958,
 10095,
 60999,
 36592,
 13293,
 17684,
 21442,
 53531,
 14157,
 6596,
 27678,
 26055,
 18271,
 61989,
 58546,
 32794,
 27006,
 58276,
 43062,
 12357,
 41484,
 36921,
 62835,
 45912,
 58937,
 34825,
 60616,
 59747,
 702,
 11522,
 59253,
 6593,
 45852,
 35043,
 17503,
 57844,
 21725,
 36602,
 43063,
 60819,
 2156,
 15400,
 22231,
 64667,
 49207,
 8917,
 40841,
 16465,
 34345,
 8678,
 39672,
 33956,
 41108,
 54850,
 31531,
 8743,
 41764,
 25932,
 25005,
 13961,
 33229,
 52350,
 56460,
 61584,
 44309,
 31823,
 64198,
 30579,
 55010,
 3793

In [48]:
def BellExperiment():
    reg = State(2) 
    HADAMARD(0,reg)
    for i in range(reg.n-1):
        CNOT(i+1,i,reg)
    return reg
measure_all(BellExperiment())

'01'

In [85]:

def EntanglementExperiment():
    reg = State(3) 
    SETVALUE(reg,1,0)
    HADAMARD(0,reg)
    CNOT(1,0,reg)
    CNOT(2,0,reg)
    return reg
measure_all(EntanglementExperiment())

'001'

In [1283]:
from datetime import date

def interpreter():
    VERSION = "0.3"
    BUILD = "Alpha"

    RULES = [
        "HADAMARD",
        "CNOT",
        "PX",
        "TOFFOLI",
        "CIRCUIT",
        "MEASURE",
        "MEASURE ALL",
        "HELP",
        "END",
        "ADD",
        "VAR",
        "POP"
    ]
    OPERATIONS = [
        "="
    ]
    DESCRIPTION = [
        "Accepts an integer i and Quantum circuit C. Performs Hadamard operation on ith qubit of Circuit C.",
        "Accepts an control, target qubit and Quantum circuit C. Flips target qubit if control qubit is |1>.",
        "Accepts an integer i and Quantum circuit C. Applies Pauli's X gate on given qubit of the Circuit C .",
        "Accepts an Quantum circuit C and an array of A,B,X values. Applies Toffoli's Gate on X qubit.",
        "Accepts an integer K. creates an quantum circuit of K qubits.",
        "Accepts an integer i and Quantum circuit C. Measures the current value of probability of ith qubit of Circuit C.",
        "Accepts an Quantum circuit C. Measures all values of all qubits in the Circuit C.",
        "Displays all the set of instructions and their description.",
        "Close the Editor",
        "Adds next instruction result to variable top of the stack.",
        "Declares a variable top of the Stack.",
        "pops the top of the variable of the Stack."
    ]

    print(f"QauntumPY {VERSION} {BUILD} | Author: ATG | {date.today()} ")
    print('Type "HELP" for all list of instructions avaliable.')
    
    instruction_stack = []
    operation_stack = []
    active_circuit = None

    while True:
        instruction = input()
        if instruction == "END":
            break
        opcodes = instruction.split(" ")
        rule = opcodes[0]

        if rule == "HADAMARD":
            pass
        elif rule == "CNOT":
            pass
        elif rule == "PX":
            pass
        elif rule == "TOFFOLI":
            pass
        elif rule == "CIRCUIT":
            pass
        elif rule == "MEASURE":
            pass
        elif rule == "MEASURE ALL":
            pass
        elif rule == "HELP":
            print("")
            for i in range(len(RULES)):
                print(RULES[i] + " :",DESCRIPTION[i])
            pass
        elif rule == "ADD":
            pass
        elif rule == "VAR":
            pass
        else:
            print()
            print(f"Invalid Instruction {rule}")
            pass



        

interpreter()

QauntumPY 0.3 Alpha | Author: ATG | 2022-04-10 
Type "HELP" for all list of instructions avaliable.

Invalid Instruction START

Invalid Instruction 


In [1281]:
file = open('firstcode.atg','w')
file.write("This will add this line")
file.close()